# How to import a CSV file into a MySQL database using python script?
#### Using iris data, you will learn how to import a CSV file into a MySQL database using python script.
===================================================================================================================

## Prerequisites

Python 3.8.3, MySQL 8.0.17, mysql-connector-python

To perform this task, you will need to:
#### 1. Prepare or identify your data
#### 2. Connect to MySQL and create a database
#### 3. Create a table
#### 4. Load the data

### Step 1: Prepare or identify your data

To begin, prepare or identify the CSV file that you'd like to import to MySQL database. For example, I loaded a iris data from sklearn.datasets.

In [4]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
iris=load_iris()
x=iris.data
y=iris.target
iris_d=np.column_stack((x, y))
irisData=pd.DataFrame(iris_d,columns=['sepalLengthcm','sepalWidthcm','petalLengthcm','petalWidthcm','species'])
irisData.loc[irisData['species']==0.0,'species']='setosa'
irisData.loc[irisData['species']==1.0,'species']='versicolor'
irisData.loc[irisData['species']==2.0,'species']='virginica'
irisData.head()

,sepalLengthcm,sepalWidthcm,petalLengthcm,petalWidthcm,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


or you can read data from your disk and import, for example, I read data from my Github account.

In [ ]:
irisData =pd.read_csv('https://github.com/Muhd-Shahid/Write-Raw-File-into-Database-Server/raw/main/iris.csv')

Where, data is the comma delimiter. Each line of the CSV file is terminated by a newline character.

### Step 2: Connect to the MySQL and create a database

Now let's create connection object to connect to MySQL server.
The connect() constructor creates a connection to the MySQL server and returns a MySQLConnection object.

In [ ]:
import mysql.connector
from mysql.connector import Error
try:
    conn = mysql.connector.connect(host='localhost',
                                         user='root',
                                         password='sql@123')
    if connection.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE irisDB")
        print("irisDB database is created")
        
except Error as e:
    print("Error while connecting to MySQL", e)

Check, did it work?

### Step 2: Create a table

First, we will insure that column and datatype parameters in the SQL table that we will create must match the number of columns and data types of the CSV file.

In [5]:
irisData.dtypes

sepalLengthcm    float64
sepalWidthcm     float64
petalLengthcm    float64
petalWidthcm     float64
species           object
dtype: object

We will create iris table under irisDB database in MySQL server.

In [ ]:
import mysql.connector
from mysql.connector import Error
try:
    conn = mysql.connector.connect(host='localhost', user='root', password='sql@123', database='irisDB')
    if conn.is_connected():
        cursor = conn.cursor()
        #print("train table")
        cursor.execute("CREATE TABLE iris(sepal_length DECIMAL(2,1) NOT NULL, sepal_width DECIMAL(2,1) NOT NULL, petal_length DECIMAL(2,1) NOT NULL, petal_width DECIMAL(2,1),species CHAR(11)NOT NULL)")
        print("iris table is created")
        
except Error as e:
    print("Error while connecting to MySQL", e)

### Step 3: Load the data: Import the CSV data into the MySQL table

### What's happening here?
#### The LOCAL keyword affects where the file is expected to be found. The file is read by the client program on the client host and sent to the server. If LOCAL is not specified, the file must be located on the server host and is read directly by the server." Reference manual
#### Since the comma is the delimiter, we use the FIELDS TERMINATED BY ','
#### Each line of the CSV file is terminated by a newline character, thus we use LINES TERMINATED BY '\n'
#### We already have the header in the MySQL table so we use IGNORE 1 ROWS

### Issues

You may face the following issues during inserting or loading csv file data directly into the table without creating any insert statement.
#### Problem: 
mysql.connector.errors.ProgrammingError: 1148 (42000): The used command is not allowed with this MySQL version
#### Solution: 
Check what value is there in your database using MySQL client by executing the command SHOW VARIABLES LIKE 'local_infile';.
if value for local_infile shows as OFF then you can set using the command set global local_infile = ON;.
#### Problem: 
The MySQL server is running with the –secure-file-priv option so it cannot execute this statement
#### Solution: 
Check what value if there for this variable using the command SHOW VARIABLES LIKE "secure_file_priv";.
If it shows NULL then you can disable security for file in my.ini file.
So open my.ini file and add the following line. my.ini file generally exists under your root folder of your MySQL installation path.